In [1]:
import pandas as pd
import numpy as np
import copy

def clean_data(data_path, remove_feats_after_ct=True, remove_TBI_rows_with_nan=True,threshold=0.5,rm_feats=True,remove_GCS_total_mismatch=True):
    # Remove rows with missing values
    data = pd.read_csv("TBI PUD 10-08-2013.csv")
    data_clean = copy.copy(data)
    # change value 92 in column to NaN for the following list of features
    list_92 = ['LocLen','SeizOccur','SeizLen','HASeverity','HAStart','VomitNbr','VomitStart','VomitLast','AMSAgitated','AMSSleep','AMSSlow','AMSRepeat','AMSOth','SFxPalpDepress','FontBulg','SFxBasHem','SFxBasOto','SFxBasPer','SFxBasRet','SFxBasRhi','HemaLoc','HemaSize','ClavFace','ClavNeck','ClavFro','ClavOcc','ClavPar','ClavTem','NeuroDMotor','NeuroDSensory','NeuroDCranial','NeuroDReflex','NeuroDOth','OSI','OSIExtremity','OSICut','OSICspine','OSIFlank','OSIAbdomen','OSIPelvis','OSIOth','Drugs','IndAge','IndAmnesia','IndAMS','IndClinSFx','IndHA','IndHema','IndLOC','IndMech','IndNeuroD','IndRqstMD','IndRqstParent','IndRqstTrauma','IndSeiz','IndVomit','IndXraySFx','IndOth','CTSed','CTSedAgitate','CTSedAge','CTSedRqst','CTSedOth','EDCT','PosCT','Finding1','Finding2','Finding3','Finding4','Finding5','Finding6','Finding7','Finding8','Finding9','Finding10','Finding11','Finding12','Finding13','Finding14','Finding20','Finding21','Finding22','Finding23']
    for i in range(len(list_92)):
        column_name = list_92[i]
        column_data = data_clean[column_name]
        #print nan indices
        indices92 = column_data[column_data == 92].index
        data_clean[column_name] = column_data.replace(92, np.nan)
        
    # change 'other' in column to NaN for the following list of features
    list_other = ['Race','EDDisposition']
    for i in range(len(list_other)):
        column_name = list_other[i]
        column_data = data_clean[column_name]
        data_clean[column_name] = column_data.replace('other', np.nan)
    # remove CTForm1
    data_clean = data_clean.drop(columns=['CTForm1'])
       
    if remove_feats_after_ct:
        # Remove features after CT
        posCT_index = data_clean.columns.get_loc('CTDone')
        data_clean = data_clean.drop(data_clean.columns[posCT_index:data_clean.shape[1]-1], axis=1)
        
    if remove_TBI_rows_with_nan:
        # Remove rows with NaN values in PosIntFinal
        data_clean = data_clean.dropna(subset=['PosIntFinal'])
    
    if rm_feats:
        # Remove features with more than threshold percent of missing values
        missing_percentage = data_clean.isnull().mean()
        missing_columns = missing_percentage[missing_percentage > threshold].index.tolist()
        data_clean = data_clean.drop(columns=missing_columns)
    
    if remove_GCS_total_mismatch:
        # remove rows where GCSTotal does not equal the sum of GCS components
        data_clean['CalculatedTotal'] = data_clean['GCSEye'] + data_clean['GCSVerbal'] + data_clean['GCSMotor']
        data_clean = data_clean[data_clean['CalculatedTotal'] == data_clean['GCSTotal']]
        data_clean = data_clean.drop(columns=['CalculatedTotal'])

    return data_clean

In [2]:
# Call the cleaning function (assuming clean_data is defined elsewhere)
data_clean = clean_data("TBI PUD 10-08-2013.csv")  # Make sure the file path is correct

# Save the cleaned DataFrame as a CSV file
data_clean_path = "cleaned_data.csv"
data_clean.to_csv(data_clean_path, index=False)

# Read the head of the saved CSV file
df = pd.read_csv(data_clean_path)  # Read the saved data back
print(df.head())

   PatNum  EmplType  Certification  InjuryMech  High_impact_InjSev  \
0       1       3.0              3        11.0                 2.0   
1       2       5.0              3         8.0                 2.0   
2       3       5.0              3         5.0                 2.0   
3       4       5.0              3         6.0                 1.0   
4       5       3.0              3        12.0                 2.0   

   Amnesia_verb  LOCSeparate  Seiz  ActNorm  HA_verb  ...  Drugs  AgeInMonth  \
0           0.0          0.0   0.0      1.0      1.0  ...    0.0         197   
1           0.0          0.0   0.0      1.0      0.0  ...    0.0          64   
2           NaN          NaN   NaN      0.0      NaN  ...    0.0         170   
3          91.0          0.0   0.0      1.0     91.0  ...    0.0          13   
4          91.0          0.0   0.0      0.0     91.0  ...    0.0          14   

   AgeinYears  AgeTwoPlus  Gender  Ethnicity  Race  Observed  EDDisposition  \
0          16      

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# Load cleaned data
dt = pd.read_csv("cleaned_data.csv")

# Verify the target column exists
target_column = 'PosIntFinal'  # Update to your target column name
if target_column not in dt.columns:
    raise ValueError(
        f"Target column '{target_column}' not found in dataset. Available columns: {list(dt.columns)}"
    )

# Split data into features (X) and target (y)
X = dt.drop(columns=target_column)
y = dt[target_column]

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display sizes of train and test sets
print(f"Training set size: {len(X_train)} samples")
print(f"Test set size: {len(X_test)} samples")

# Check original class distribution in training set
print("Original training set class distribution:")
print(y_train.value_counts())

# Handle class imbalance by oversampling the minority class
# Combine X_train and y_train into one dataset
train_data = np.hstack((X_train, y_train.values.reshape(-1, 1)))

# Separate majority and minority classes
majority = train_data[train_data[:, -1] == 0]  # Assuming class 0 is majority
minority = train_data[train_data[:, -1] == 1]  # Assuming class 1 is minority

# Oversample the minority class
minority_oversampled = resample(minority, replace=True, n_samples=len(majority), random_state=42)

# Combine back into a balanced dataset
train_balanced = np.vstack((majority, minority_oversampled))
np.random.shuffle(train_balanced)  # Shuffle the dataset

# Split features and target
X_train_balanced = train_balanced[:, :-1]
y_train_balanced = train_balanced[:, -1]

# Display the size and class distribution of the balanced training set
print(f"Balanced training set size: {len(X_train_balanced)} samples")
print("Balanced training set class distribution:")
print(np.bincount(y_train_balanced.astype(int)))

Training set size: 33628 samples
Test set size: 8408 samples
Original training set class distribution:
0.0    33065
1.0      563
Name: PosIntFinal, dtype: int64
Balanced training set size: 66130 samples
Balanced training set class distribution:
[33065 33065]


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn imports
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    confusion_matrix, 
    classification_report, 
    roc_curve, 
    roc_auc_score
)

def evaluate_fnr(y_true, y_pred):
    """
    Calculate the False Negative Rate (FNR).
    
    Parameters:
    -----------
    y_true : array-like
        True labels
    y_pred : array-like
        Predicted labels

    Returns:
    --------
    float: False Negative Rate
    """
    cm = confusion_matrix(y_true, y_pred)
    fn = cm[1, 0]  # False negatives
    tp = cm[1, 1]  # True positives
    return fn / (fn + tp) if (fn + tp) > 0 else 0.0

def advanced_logistic_regression(dt, target_column):
    """
    Advanced Logistic Regression with comprehensive preprocessing and model evaluation
    
    Parameters:
    -----------
    dt : pandas.DataFrame
        Input dataset
    target_column : str
        Name of the target variable column
    
    Returns:
    --------
    dict: A comprehensive dictionary of model performance metrics and insights
    """
    # Separate features and target
    X = dt.drop(columns=[target_column])
    y = dt[target_column]
    
    # Identify numeric and categorical columns
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    
    # Create preprocessing pipelines
    preprocessor = ColumnTransformer(
        transformers=[('num', Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', RobustScaler())  # More robust scaling
            ]), numeric_features)
        ])
    
    # Full pipeline with feature selection and logistic regression
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('feature_selection', SelectKBest(f_classif, k=10)),  # Select top 10 features
        ('classifier', LogisticRegression(
            max_iter=5000,  # Increased max iterations
            class_weight='balanced',  # Handle class imbalance
            random_state=42
        ))
    ])
    
    # Hyperparameter tuning grid
    param_grid = {
        'feature_selection__k': [5, 10, 15],  # Different numbers of top features
        'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'classifier__penalty': ['l1', 'l2'],
        'classifier__solver': ['liblinear', 'saga']
    }
    
    # Stratified K-Fold Cross Validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Grid Search with Cross-Validation
    grid_search = GridSearchCV(
        pipeline, 
        param_grid, 
        cv=cv, 
        scoring='f1',  # Using F1 score for balanced evaluation
        n_jobs=-1,  # Use all available cores
        verbose=1
    )
    
    # Fit the grid search
    grid_search.fit(X, y)
    
    # Best model
    best_model = grid_search.best_estimator_
    
    # Predictions
    y_pred = best_model.predict(X)
    
    # Probability for the positive class
    y_proba = best_model.predict_proba(X)[:, 1]  # Probabilities for the positive class
    
    # Performance Metrics
    performance_metrics = {
        'best_params': grid_search.best_params_,
        'accuracy': accuracy_score(y, y_pred),
        'precision': precision_score(y, y_pred, average='weighted'),
        'recall': recall_score(y, y_pred, average='weighted'),
        'f1_score': f1_score(y, y_pred, average='weighted'),
        'roc_auc': roc_auc_score(y, y_proba),  # Using probabilities for AUC
        'false_negative_rate': evaluate_fnr(y, y_pred)  # False Negative Rate
    }
    
    # Learning Curve
    train_sizes, train_scores, test_scores = learning_curve(
        best_model, X, y, 
        cv=cv, 
        train_sizes=np.linspace(0.1, 1.0, 10),
        scoring='f1_weighted'
    )
    
    # Visualization of Learning Curve
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training score')
    plt.plot(train_sizes, np.mean(test_scores, axis=1), label='Cross-validation score')
    plt.title('Learning Curve')
    plt.xlabel('Training Examples')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.tight_layout()
    plt.savefig('learning_curve.png')
    plt.close()
    
    # Confusion Matrix
    cm = confusion_matrix(y, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    # Cross-validation scores
    cv_scores = cross_val_score(best_model, X, y, cv=cv, scoring='f1_weighted')
    
    # Additional insights
    performance_metrics.update({
        'cross_val_scores': cv_scores,
        'mean_cv_score': np.mean(cv_scores),
        'std_cv_score': np.std(cv_scores)
    })
    
    # Print detailed classification report
    print("\nDetailed Classification Report:")
    print(classification_report(y, y_pred))
    
    return performance_metrics, y_proba

# Usage example (uncomment and modify as needed)
results, probabilities = advanced_logistic_regression(dt, 'PosIntFinal')
print(results)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


/usr/local/linux/mambaforge-3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/linux/mambaforge-3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/linux/mambaforge-3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/linux/mambaforge-3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/linux/mambaforge-3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/linu


Detailed Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98     41319
         1.0       0.21      0.63      0.31       717

    accuracy                           0.95     42036
   macro avg       0.60      0.80      0.64     42036
weighted avg       0.98      0.95      0.96     42036

{'best_params': {'classifier__C': 0.01, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear', 'feature_selection__k': 5}, 'accuracy': 0.95246931201827, 'precision': 0.9799931177214962, 'recall': 0.95246931201827, 'f1_score': 0.9640763660133603, 'roc_auc': 0.8044821049599362, 'false_negative_rate': 0.3668061366806137, 'cross_val_scores': array([0.96329176, 0.96404425, 0.9671322 , 0.96750861, 0.96269169]), 'mean_cv_score': 0.9649337017482589, 'std_cv_score': 0.001998864414177428}


In [6]:
print("False Negative Rate:", results['false_negative_rate'])

False Negative Rate: 0.3668061366806137


In [15]:
## Save the Probabilities model

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Create a preprocessing pipeline to handle missing values and scaling
preprocessing_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # Replace NaNs with median of each column
    ('scaler', StandardScaler())  # Scale the features
])

# Preprocess the training data
X_train_balanced_cleaned = preprocessing_pipeline.fit_transform(X_train_balanced)
y_train_balanced_cleaned = y_train_balanced

# Preprocess the test data using the same transformations
X_test_cleaned = preprocessing_pipeline.transform(X_test)

# Initialize the Logistic Regression model
logreg_model = LogisticRegression()

# Fit the model on the preprocessed training data
logreg_model.fit(X_train_balanced_cleaned, y_train_balanced_cleaned)

# Predictions for Logistic Regression
y_pred_logreg = logreg_model.predict(X_test_cleaned)
y_proba_logreg = logreg_model.predict_proba(X_test_cleaned)[:, 1]  # Probabilities for the positive class

# Save the model probabilities
np.save('logreg.npy', y_proba_logreg)

/usr/local/linux/mambaforge-3.11/lib/python3.11/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


In [ ]:
from sklearn.metrics import recall_score

y_test = test_df["PosIntFinal"]
X_test = test_df.drop(columns=["PosIntFinal"])

y_test_prob = model.predict(X_test)

y_test_pred = (y_test_prob > 0.5).astype("int32")

# Step 3: Calculate recall
recall = recall_score(y_test, y_test_pred)
print(f"Recall for the test set: {recall:.4f}")